In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.imports import *
from lib.video_processor import VideoProcessor
from lib.misc import *

In [4]:
interpolate_rate=5
eyes_int_rate=1
eye_model_size=256
input_source = 0 # "example_video/example_vid_original.mp4"
form="onnx"


eye_batch_rate=(interpolate_rate//(eyes_int_rate+1))+1
convert_pt_to_onnx(model_path=f'models/eyes_{eye_model_size}_auto.pt', imgsz=eye_model_size, batch=eye_batch_rate, form=form)

processor = VideoProcessor(eye_model_size=eye_model_size, form=form)
out = processor.start(video_path=input_source, interpolate_rate=interpolate_rate, eyes_int_rate=eyes_int_rate, eye_batch_rate=eye_batch_rate, return_instead=False, disable_stats=False);

Ultralytics YOLOv8.2.9 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0
YOLOv8n-pose summary (fused): 187 layers, 3154073 parameters, 0 gradients, 8.6 GFLOPs

PyTorch: starting from 'models/eyes_256_auto.pt' with input shape (3, 3, 256, 256) BCHW and output shape(s) (3, 35, 1344) (6.2 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.5s, saved as 'models/eyes_256_auto.onnx' (12.1 MB)

Export complete (1.8s)
Results saved to /home/tobi/Desktop/eye_tracking/models
Predict:         yolo predict task=pose model=models/eyes_256_auto.onnx imgsz=256  
Validate:        yolo val task=pose model=models/eyes_256_auto.onnx imgsz=256 data=data/yolo_pose_dataset.yaml  
Visualize:       https://netron.app
Loading models/yolov8n-face-lindevs.onnx for ONNX Runtime inference...
Loading models/eyes_256_auto.onnx for ONNX Runtime 

In [ ]:
# write_video(out, "example_video/flexpressions_cropped.mp4")

## Training

In [24]:
prepare_video_annotations_for_training(val_ratio=0.1, multiply_train_data=32)

Conversion completed.


In [ ]:
# if you don't want to track training with comet don't run this cell and set usecomet=False in the next cell
init_comet(api_key="your_api_key")

In [ ]:
for image_size in [128, 256, 64, 512]: # might get stuck before starting 2nd batch, if this happens just restart this cell
    train_model(pretrained_model='models/yolov8n-pose.pt', save_name=f"training_{image_size}_autotrain", imgsz=image_size, epochs=100, batch=256, amp=True, workspace="comet_workspacename", usecomet=False)

New https://pypi.org/project/ultralytics/8.2.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.8 🚀 Python-3.11.9 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
engine/trainer: task=pose, mode=train, model=YOLO(
  (model): PoseModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats

train: Scanning /home/tobi/ai/projects/eye_tracking/data/03_train_split... 3840 


train: New cache created: /home/tobi/ai/projects/eye_tracking/data/03_train_split.cache


train: Caching images (0.1GB RAM): 100%|██████████| 3840/3840 [00:02<00:00, 1909
val: Scanning /home/tobi/ai/projects/eye_tracking/data/03_val_split... 13 images

val: New cache created: /home/tobi/ai/projects/eye_tracking/data/03_val_split.cache


...


      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     67/100      2.16G     0.8584     0.9023     0.3025     0.5308      0.678   
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all         13         13      0.993          1      0.995      0.511      0.993          1      0.995      0.833



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     68/100      2.16G     0.8672     0.9303     0.2954     0.5187     0.6766   